LIBRERÍAS

In [1]:
import pandas as pd
from datetime import date, timedelta, datetime
from google.colab import files

CARGA DE DATOS Y AJUSTES INICIALES

In [2]:
url_datos_1 = 'https://raw.githubusercontent.com/rgarciamartin/TFM-BigData/main/Datos/H1.csv?token=GHSAT0AAAAAABWGXEHQF4TKJKBQN7QTQ3J4YWRTR5A'
url_datos_2 = 'https://raw.githubusercontent.com/rgarciamartin/TFM-BigData/main/Datos/H2.csv?token=GHSAT0AAAAAABWGXEHQZ6ESF5GYQGHUZAOWYWRTSLA'

In [3]:
df_inicial_1 = pd.read_csv(url_datos_1)
df_inicial_1['hotel'] = 'Resort Hotel'

df_inicial_2 = pd.read_csv(url_datos_2)
df_inicial_2['hotel'] = 'City Hotel'

df_inicial = pd.concat([df_inicial_1, df_inicial_2], ignore_index=True)

df_inicial

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,hotel
0,0,342,2015,July,27,1,0,0,2,0.0,...,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,Resort Hotel
1,0,737,2015,July,27,1,0,0,2,0.0,...,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,Resort Hotel
2,0,7,2015,July,27,1,0,1,1,0.0,...,NULL,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,Resort Hotel
3,0,13,2015,July,27,1,0,1,1,0.0,...,304,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,Resort Hotel
4,0,14,2015,July,27,1,0,2,2,0.0,...,240,NULL,0,Transient,98.00,0,1,Check-Out,2015-07-03,Resort Hotel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0,23,2017,August,35,30,2,5,2,0.0,...,394,NULL,0,Transient,96.14,0,0,Check-Out,2017-09-06,City Hotel
119386,0,102,2017,August,35,31,2,5,3,0.0,...,9,NULL,0,Transient,225.43,0,2,Check-Out,2017-09-07,City Hotel
119387,0,34,2017,August,35,31,2,5,2,0.0,...,9,NULL,0,Transient,157.71,0,4,Check-Out,2017-09-07,City Hotel
119388,0,109,2017,August,35,31,2,5,2,0.0,...,89,NULL,0,Transient,104.40,0,0,Check-Out,2017-09-07,City Hotel


In [4]:
df_inicial.rename(columns={
    'IsCanceled' : 'is_canceled',
    'LeadTime' : 'lead_time',
    'ArrivalDateYear' : 'arrival_date_year',
    'ArrivalDateMonth' : 'arrival_date_month',
    'ArrivalDateWeekNumber' : 'arrival_date_week_number',
    'ArrivalDateDayOfMonth' : 'arrival_date_day_of_month',
    'StaysInWeekendNights' : 'stays_in_weekend_nights',
    'StaysInWeekNights' : 'stays_in_week_nights',
    'Adults' : 'adults',
    'Children' : 'children',
    'Babies' : 'babies',
    'Meal' : 'meal',
    'Country' : 'country',
    'MarketSegment' : 'market_segment',
    'DistributionChannel' : 'distribution_channel',
    'IsRepeatedGuest' : 'is_repeated_guest',
    'PreviousCancellations' : 'previous_cancellations',
    'PreviousBookingsNotCanceled' : 'previous_booking_not_canceled',
    'ReservedRoomType' : 'reserved_room_type',
    'AssignedRoomType' : 'assigned_room_type',
    'BookingChanges' : 'booking_changes',
    'DepositType' : 'deposit_type',
    'Agent' : 'agent',
    'Company' : 'company',
    'DaysInWaitingList' : 'days_in_waiting_list',
    'CustomerType' : 'customer_type',
    'ADR' : 'adr',
    'RequiredCarParkingSpaces' : 'required_car_parking_spaces',
    'TotalOfSpecialRequests' : 'total_of_special_requests',
    'ReservationStatus' : 'reservation_status',
    'ReservationStatusDate' : 'reservation_status_date'
}, inplace=True)

Nuevas Variables
- Fecha de llegada completa en formato string
- Fecha de llegada completa en formato fecha
- Fecha de cambio de estado en formato fecha
- Diferencia en días entre las 2 fechas anteriores:
  - Sí el resultado es negativo, implica que la reserva se canceló antes de llegada (el valor del número serían los días de antelación con la que se canceló)
  - Si el resultado es positivo, indica los días de estancia desde la llegada hasta el desalojo del hotel

In [5]:
# Fecha de llegada completa en formato fecha
arrival_date_list = []
for item in range(0, len(df_inicial), 1):
    if df_inicial['arrival_date_day_of_month'][item] < 10:
        arrival_date = datetime.strptime('{}-{}-0{}'.format(df_inicial['arrival_date_year'][item],df_inicial['arrival_date_month'][item],df_inicial['arrival_date_day_of_month'][item]),
                                "%Y-%B-%d"
                            )
    else:
        arrival_date = datetime.strptime('{}-{}-{}'.format(df_inicial['arrival_date_year'][item],df_inicial['arrival_date_month'][item],df_inicial['arrival_date_day_of_month'][item]),
                                "%Y-%B-%d"
                            )
    arrival_date_list.append(arrival_date)

df_inicial['arrival_date_complete'] = arrival_date_list

df_inicial['arrival_date_complete']

0        2015-07-01
1        2015-07-01
2        2015-07-01
3        2015-07-01
4        2015-07-01
            ...    
119385   2017-08-30
119386   2017-08-31
119387   2017-08-31
119388   2017-08-31
119389   2017-08-29
Name: arrival_date_complete, Length: 119390, dtype: datetime64[ns]

In [6]:
# Fecha de llegada completa en formato string
arrival_date_list_str = []
for date in arrival_date_list:
    arrival_date_list_str.append(date.strftime("%Y-%B-%d"))

df_inicial['arrival_date_complete_str'] = arrival_date_list_str

df_inicial['arrival_date_complete_str']

0           2015-July-01
1           2015-July-01
2           2015-July-01
3           2015-July-01
4           2015-July-01
               ...      
119385    2017-August-30
119386    2017-August-31
119387    2017-August-31
119388    2017-August-31
119389    2017-August-29
Name: arrival_date_complete_str, Length: 119390, dtype: object

In [7]:
# Fecha de cambio de estado en formato fecha
reservation_status_date_list = []
for item in df_inicial['reservation_status_date']:
    reservation_date = datetime.strptime(item, "%Y-%m-%d")
    reservation_status_date_list.append(reservation_date)
    
df_inicial['reservation_status_date_datetime'] = reservation_status_date_list

df_inicial['reservation_status_date_datetime']

0        2015-07-01
1        2015-07-01
2        2015-07-02
3        2015-07-02
4        2015-07-03
            ...    
119385   2017-09-06
119386   2017-09-07
119387   2017-09-07
119388   2017-09-07
119389   2017-09-07
Name: reservation_status_date_datetime, Length: 119390, dtype: datetime64[ns]

In [8]:
# Diferencia en días entre la fecha de llegada completa y la fecha de cambio de estado
df_inicial['arrival_and_reservation_date_diff'] = df_inicial['reservation_status_date_datetime'] - df_inicial['arrival_date_complete']

df_inicial['arrival_and_reservation_date_diff'] = [item.days for item in df_inicial['arrival_and_reservation_date_diff']]

df_inicial['arrival_and_reservation_date_diff']

0         0
1         0
2         1
3         1
4         2
         ..
119385    7
119386    7
119387    7
119388    7
119389    9
Name: arrival_and_reservation_date_diff, Length: 119390, dtype: int64

La siguiente celda sirve para descargar el los datos que quieras en formato csv en local

df_inicial.to_csv("inicial.csv")
files.download("inicial.csv")